In [1]:
import random
from colorama import Fore, Back

### Classe Deck

In [2]:
class Deck():
    def __init__(self):
        """
        Cria um baralho totalmente aleatório
        """

        self.cartas_deck = [str(i) + f for i in range(1, 10 + 1) 
                            for f in ["♠", "♥", "♦", "♣"]]
        
        self.cartas_deck += [str(i) + f for i in ["K", "Q", "J"]
                            for f in ["♠", "♥", "♦", "♣"]]
        random.shuffle(self.cartas_deck)

    def dar_cartas(self):
        """
        Retorna uma carta aleátoria
        """

        dar_cartas_t = True
        while dar_cartas_t:
            if len(self.cartas_deck) >= 1:
                return self.cartas_deck.pop(0)
            else:
                print("*Embaralhando um novo deck*")
                self.__init__()
                continue

### Classe Jogador

In [3]:
class Jogador(Deck):
    def __init__(self, balance=0):
        self.balance = balance
        print("Banca total:", Fore.GREEN + f"R${self.balance}", Fore.RESET)
        self.cartas = []
        self.lista_cartas_pegas = []
        self.lista_2 = []
        self.resultado_soma = 0

    def novo_jogo(self):
        """
        Reseta o inventário do jogador.
        """
        self.cartas = []
        self.lista_cartas_pegas = []
        self.lista_2 = []
        self.resultado_soma = 0

    def deposit(self, quantity):
        """
        Deposita a quantia na balance do jogador, retornando as
        informações do depósito.
        """
        
        self.balance += quantity
        print("Depósito de:", Fore.GREEN + f"R${quantity}",
              Fore.RESET + "Banca total:", Fore.GREEN + f"R${self.balance}",
              Fore.RESET)

    def receber_carta(self, carta):
        """
        Adiciona carta a lista self.cartas.
        """
        self.cartas.append(carta)

    def subtrair(self, valor):
        """
        Subtrai da balance do jogador um valor.
        """
        self.balance -= valor
        print(" ")
        return self.balance

    def ganhar(self, valor):
        """
        Adiciona um valor à balance do jogador .
        """
        self.balance += valor

    def soma_cartas(self):
        """
        Soma as cartas do jogador, retornando o resultado da soma.
        
        Caso na lista self.lista_2, tenha o valor 1, e a soma da lista + 10
        menor ou igual a 21, adiciona mais 10, como no jogo original.
        """
        for x in self.cartas:
            if x not in self.lista_cartas_pegas:
                self.lista_cartas_pegas.append(x)
                try:
                    valor = int(x[0:2])  # 10
                    self.lista_2.append(int(x[0:2]))
                except:
                    try:
                        valor = int(x[0])  # 1 - 9
                        self.lista_2.append(valor)
                    except:
                        valor = 10  # K, Q, J
                        self.lista_2.append(valor)

                if 1 in self.lista_2 and sum(self.lista_2) + 10 <= 21:
                    self.resultado_soma = sum(self.lista_2) + 10
                else:
                    self.resultado_soma = sum(self.lista_2)
            else:
                continue

        return self.resultado_soma

### Classe Deller

In [4]:
class Deller(Jogador, Deck):
    """
    Define o Deller.
    """
 
    def __init__(self, balance=0):
        self.balance = balance
        self.cartas = []
        self.lista_cartas_pegas = []
        self.lista_2 = []
        self.resultado_soma = 0

### Defs

###### Deposito inicial

In [5]:
def deposito_inicial():
    global jogador1, deller, baralho, jogar_principal
    
    while jogar_principal:

        try:
            depositar = int(input("Digite seu depósito inicial: "))
            print(" ")

        except:
            print(" ")
            print("Digite um valor válido")
            continue
        
        jogador1 = Jogador(depositar)
        deller = Deller()
        baralho = Deck()
        print('-' * 20)
        jogar_principal = False

###### Input aposta pelo jogador. 
Cheat: use & na frente do valor para não desconta-lo na sua banca, dinheiro
ilimitado.

In [6]:
def input_aposta():
    
    global aposta, jogador1, deller, ultima_aposta
    

    jogador1.novo_jogo()  #Reseta o inventario do jogador
    deller.novo_jogo()  #Reseta o inventario do deller
    
    aposta = input("Digite sua aposta ou ENTER para a ultima: ")
    try:
        aposta = int(aposta)
        ultima_aposta = aposta
    except:
        try:
            if "$" in aposta:
                aposta = int(aposta[1:])
                jogador1.deposit(aposta)
                ultima_aposta = aposta
            else:
                aposta = ultima_aposta
        except:
            aposta = ultima_aposta

    if jogador1.balance >= aposta:
        jogador1.subtrair(aposta)
        print("Aposta de:", Fore.GREEN + f"R${aposta}" + "" + Fore.RESET)
        print(" \n" * 50)
    else:
        print(" \nAposta maior que sua banca.")
        print("Sendo assim, colocaremos sua aposta igual a sua banca")
        print('-' * 50)
        aposta = jogador1.balance
        jogador1.subtrair(aposta)

    for dar_cartas_duas_vezes in range(2):  # da as cartas
        deller.receber_carta(baralho.dar_cartas())
        jogador1.receber_carta(baralho.dar_cartas())

###### Imprime a mesa
Nela se coloca a função 'print_mensagens_fim'

In [7]:
def print_mesa(mensagem):
    global escolhas, continuar_jogo
    
    print("-"*30)
    if checar:
        print("Mão deller:", deller.cartas, "Soma:", deller.soma_cartas())
        mensagem()
        print(" ")
        escolhas = False
        continuar_jogo = True
    else:
        print("Mão deller:", deller.cartas[0], " 🂠")
        print(" \n"*2)
        escolhas = True
    print("Sua mão:", jogador1.cartas, " Soma:", jogador1.soma_cartas(),
          " Aposta:",
          Fore.GREEN + f"R${aposta}" + Fore.RESET + " Banca total:",
          Fore.GREEN + f"R${jogador1.balance}" + Fore.RESET)

###### Imprime mensagem final
Imprime a mesa e uma mensagem do fim de jogo, como PUSH, Você perdeu,
ou Você ganhou

In [8]:
def mensagens():

    def push():
        print("Push.")
        
    def bust():
        print("BUST:", Fore.RED + f"R${aposta}" + Fore.RESET)
        
    def you_win():
        print("Você ganhou:", Fore.GREEN + f"R${aposta}" + Fore.RESET)
    
    def deller_win():
        print("Deller win:", Fore.RED + f"R${aposta}" + Fore.RESET)
        
    #====================================================================
    
    
    if jogador1.soma_cartas() > 21: #Bust
        return bust()
    
    elif jogador1.soma_cartas() == deller.soma_cartas(): #Push
        jogador1.ganhar(aposta)
        return push()
    
    elif jogador1.soma_cartas() > 21: #Bust
        return bust()

    elif jogador1.soma_cartas() > deller.soma_cartas() or deller.soma_cartas() > 21: # You win
        jogador1.ganhar(aposta * 2)
        return you_win()

    elif deller.soma_cartas() > jogador1.soma_cartas() and deller.soma_cartas() <= 21:
        return deller_win()


###### ESCOLHAS + CHECKER

In [9]:
def print_escolhas():

    global decisao, escolhas, checar
    
    escolhas = True
    
    while escolhas:
        try:
            decisao = int(
                input(Back.GREEN + "1 - Hit" + Back.RED +
                      "2 - Stay" + Back.YELLOW + "3 - Double" +
                      "" + Back.RESET))
            escolhas = False
        except:
            continue
            
        if decisao in range(1,3+1):
            if decisao == 1: # HIT
                print(" \n"*50)
                jogador1.receber_carta(baralho.dar_cartas())
                
                if jogador1.soma_cartas() > 21:
                    tabela = True
                    checar = True 
                    
                else:    
                    tabela = True

            elif decisao == 2: # STAY - ok
                if deller.soma_cartas() < 17: # Da cartas ao deller
                    while deller.soma_cartas() < 17:
                        deller.receber_carta(baralho.dar_cartas()) 
                
                print(" \n"*50)
                tabela = True
                checar = True
                
            else: # DOUBLE - ok
                if deller.soma_cartas() < 17: # Da cartas ao deller
                    while deller.soma_cartas() < 17:
                        deller.receber_carta(baralho.dar_cartas())
                        
                if aposta * 2 <= jogador1.balance:
                    jogador1.subtrair(aposta)
                    aposta *= 2
                    print(" \n"*50)
                    tabela = True
                    checar = True
                else:
                    print("Sem caixa para isso.")
                    continue
        else:
            print(" \n"*50)
            continue
        
        escolhas = False

###### Continuar jogo

In [10]:
def continuar():
    global continuar_jogo, apostar, tabela, jogar_principal, checar
    

    print("-" * 30)
    continuar_jogando = input("Enter para continuar, outra coisa para parar")
    print("-" * 30)

    if continuar_jogando == "":
        apostar = True
        continuar_jogo = False
        tabela = False
        checar = False
        
    else:
        jogar_principal = False
        continuar_jogo = False
        tabela = False
        apostar = False
        

### ------------------------ Jogo -------------------------

In [ ]:
if __name__ == "__main__":

    jogar_principal = True
    apostar = True
    continuar_jogo = False
    decisao_stay = True
    checar = False
    ultima_aposta = 0

    while jogar_principal == True:
        deposito_inicial()
        apostar = True

        while apostar == True:
            input_aposta()
            tabela = True
        # ======================================================    
            while tabela == True:      #            /\    
                print_mesa(mensagens)  #            |
                                       #            |         |
                while escolhas == True:#-----If HIT|-----Else |
                    print_escolhas()   #                      V
        # ======================================================             
                while continuar_jogo:
                    continuar()

Digite seu depósito inicial: 1000
 
Banca total: R$1000 
--------------------
Digite sua aposta ou ENTER para a ultima: 100
 
Aposta de: R$100
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

------------------------------
Mão deller: 9♥  🂠
 
 

Sua mão: ['8♣', 'J♦']  Soma: 18  Aposta: R$100 Banca total: R$900
1 - Hit2 - Stay3 - Double2
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

------------------------------
Mão deller: ['9♥', '9♠'] Soma: 18
Push.
 
Sua mão: ['8♣', 'J♦']  Soma: 18  Aposta: R$100 Banca total: R$1000
------------------------------
Enter para continuar, outra coisa para parar
------------------------------
Digite sua aposta ou ENTER para a ultima: 
 
Aposta de: R$100
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

------------------------------
Mão deller: K♣  🂠
 
 

Sua mão: ['3♦', '5♠']  Soma: 8  Aposta: R$10